<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220919_%EC%A7%88%EB%AC%B8%EC%97%90_%EB%8B%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 질의 응답 과제
질의 응답(Question Answering) 이란 질문에 답하는 과지입니다. 이 창에서는 LG CNS가 공개한 KorQUAD v.1 데이터셋을 사용


## 7-1 질의응답 모델 훑어보기
  - 질의 응답(Question Answering)이란 질문에 답을 하는 과제
  - 질의 응답 과제의 유형은 다양
  - 다음 과제에서 진행하는 질의 응답 과제는 질문에 대한 답을 지문에서 찾는것
  

In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 641 kB/s 
     |████████████████████████████████| 57 kB 3.5 MB/s 
     |████████████████████████████████| 582 kB 10.2 MB/s 
     |████████████████████████████████| 2.8 MB 53.8 MB/s 
     |████████████████████████████████| 419 kB 74.7 MB/s 
     |████████████████████████████████| 880 kB 57.7 MB/s 
     |████████████████████████████████| 120 kB 59.8 MB/s 
     |████████████████████████████████| 3.3 MB 45.6 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=66f8c5baa789a8ca35c10b892564e5dab3eb1175a8642be896527dfa55217d8f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstallin

In [2]:
# 구글드라이브와 연결
from google.colab  import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!mkdir /content/Korpora
!mkdir /content/Korpora/korquad-v1/
!wget -O /content/Korpora/korquad-v1/KorQuAD_v1.0_train.json https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget -O /content/Korpora/korquad-v1/KorQuAD_v1.0_dev.json https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json

--2022-09-20 05:45:43--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘/content/Korpora/korquad-v1/KorQuAD_v1.0_train.json’

/content/Korpora/ko 100%[===================>]  36.74M  83.0MB/s    in 0.4s    

2022-09-20 05:45:46 (83.0 MB/s) - ‘/content/Korpora/korquad-v1/KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-09-20 05:45:46--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to:

In [4]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_corpus_name="korquad-v1",
    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa',
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32,
    learning_rate=5e-5,
    epochs=1,
    tpu_cores=0,
    seed=7,
)

### 랜덤시드 고정

In [5]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


### 로거 설정
- 각종 로그들을 출럭하는 로거를 설정

In [6]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)
INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e

## 토크나이저 준비하기
- 이준범님이 공개하신 kcbert-base 모델이 사용하는 토크나이저를 선언

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False, # 소문자 변환여부 한글이라 False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [8]:
import json

json.load(open('/content/Korpora/korquad-v1/KorQuAD_v1.0_dev.json',
               'r', encoding='utf-8'))['data'][0]['paragraphs'][0:2]

[{'qas': [{'answers': [{'text': '1989년 2월 15일', 'answer_start': 0}],
    'id': '6548850-0-0',
    'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
   {'answers': [{'text': '임수경', 'answer_start': 125}],
    'id': '6548850-0-1',
    'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
   {'answers': [{'text': '1989년', 'answer_start': 0}],
    'id': '6548853-0-0',
    'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
   {'answers': [{'text': '학생회관 건물 계단', 'answer_start': 365}],
    'id': '6548853-0-1',
    'question': '임종석을 검거한 장소는 경희대 내 어디인가?'},
   {'answers': [{'text': '서울지방경찰청 공안분실', 'answer_start': 457}],
    'id': '6548853-0-2',
    'question': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
   {'answers': [{'text': '임종석', 'answer_start': 87}],
    'id': '6332405-0-0',
    'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
   {'answers': [{'text': '여의도 농민 폭력 시위', 'answer_start': 13}],
    'id': '6332405-0-1',
    'question': '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?'}],
  '

In [18]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 17.806 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 17.806 s]


In [10]:
train_dataset[0]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 8601, 4633, 29697, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 4104, 4027, 8793, 13985, 391, 9132, 4113, 10966, 11728, 12023, 14657, 4091, 8598, 16639, 341, 4573, 4771, 4027, 2139, 8478, 14416, 214, 8202, 17, 2451, 13007, 1480, 4313, 4538, 4008, 8601, 4633, 22903, 4113, 1676, 868, 4913, 7965, 1789, 4203, 4110, 15031, 786, 250, 4057, 10878, 4007, 2593, 4094, 4128, 10289, 4113, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:
train_dataset[1]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 4170, 4042, 341, 4573, 4771, 28, 4566, 4027, 10599, 18907, 208, 9504, 24835, 15, 11060, 2451, 4780, 4032, 18548, 4113, 3231, 23243, 4104, 4042, 1843, 4771, 7965, 28987, 4153, 2451, 15489, 4113, 13928, 17283, 575, 4261, 26783, 8114, 8852, 9107, 4082, 28498, 8131, 17, 8225, 4042, 1114, 4281, 4194, 17138, 4042, 9961, 8222, 14041, 10892, 4113, 2524, 4443, 8032, 12710, 21602, 18625, 24569, 4136, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

- train_dataset.input_ids: 원본 문장을 토큰화한 결과
- train_dataset.attention_mask: 해당 위치의 토큰이 패딩(0)인지 아닌지(1)
- train_dataset.token_type_ids:[CSL]질문[SEP]에 해당하는 세그먼트는0 지문에해당하는 두번째 세그먼트는1, 나머지 패딩에 속하는 세그먼트는 0

In [12]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

- sampler : 샘플링 방식을 정의
- collate_fn: 인스턴스를 배치로 만드는 역할

## 평가용 데이터 로더 구축

In [13]:
# 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [00:16<00:00, 333.60it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##위반 ) 으로 지명 ##수 ##배 ##되었다 . 198 ##9년 3월 12 ##일 서울 ##지 ##방 ##검찰 ##청 공안 ##부는 임종석 ##의 사전 ##구속 ##영장 ##을 발부 ##받았 ##다 . 같은 해 6월 30 ##일 평양 ##축 ##전에 임 ##수 ##경을 대표로 파견 ##하여 국가보안법 ##위반 혐의가 추가 ##되었다 . 경찰은 12월 18 ##일 ~ 20 ##일 사이 서울 경 ##희 ##대학 ##교 ##에서 임종석 ##이 성 ##명 발표 ##를 추진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18 ##일 오전 7시 40 ##분 경 가스 ##총 ##과 [SEP]
INFO:ratsnlp:question & context: [CLS] 198 ##9년 6월 30 ##일 평양 ##축 ##전에 대표로 파견 된 인물 ##은 ? [SEP] 198 ##9년 2월 15일 여의도 농민 폭력 시위를 주도 ##한 혐의 ( 폭력 ##행위 ##등 ##처벌 ##에 ##관 ##한 ##법 ##률 ##

https://ratsgo.github.io/nlpbook/docs/qa/detail/
다른 데이터셋을 고치려면 qa 들어가서 다른모델 사용하기를 참고

## 모델 불러오기

### 모델 초기화
- BertForQuestionAnswering은 프리트레인을 마친 BERT 모델

In [19]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    'beomi/kcbert-base',
)
model = BertForQuestionAnswering.from_pretrained(
        'beomi/kcbert-base',
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

## 모델 학습시키기
  - 파이토치 라이트닝이 제공하는 라이트닝 모듈을 상속받아 태스크를 정의/

In [20]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [21]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [22]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
!pip install -U --no-cache-dir gdown --pre
print("Downloading...")
!gdown {"1-TtX8FfhVzVCQJPFUyeue_vDLtzcDfLI"} -O /content/drive/MyDrive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.45.ckpt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1-TtX8FfhVzVCQJPFUyeue_vDLtzcDfLI 



In [ ]:
# !mkdir /content/drive/MytDrive/nlpbook
# !mkdir /content/drive/MytDrive/nlpbook/checkpoint-qa

mkdir: cannot create directory ‘/content/drive/MytDrive/nlpbook’: No such file or directory
mkdir: cannot create directory ‘/content/drive/MytDrive/nlpbook/checkpoint-qa’: No such file or directory


In [23]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa',
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.45.ckpt


In [27]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

### 체크포인트 로드

In [28]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

### BERT 설정, 모델 초기화 진행

In [29]:
# BERT 설정 로드
from transformers import BertConfig
from transformers import BertForQuestionAnswering

pretrained_model_config =BertConfig.from_pretrained(
    args.pretrained_model_name,
)
model = BertForQuestionAnswering(pretrained_model_config)

In [30]:
# 체크포인트 읽기
model.load_state_dict({k.replace('model.',""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [31]:
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [32]:
def inference_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens=False,
            truncation=True,
            max_length=32
       )
        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair=context,
            truncation="only_second",
            padding="max_length",
            max_length=128,
            return_token_type_ids=True,
        )
        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=-1).item()
            end_pred = outputs.end_logits.argmax(dim=-1).item()
            pred_text = tokenizer.decode(inputs['input_ids']\
                                         [start_pred:end_pred+1])
    else:
        pred_text = ""
    return {
        'question': question,
        'context': context,
        'answer': pred_text,
    }

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://81fb-104-199-127-55.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [40]:
inference_fn('개통된 년도는 언제인가?','한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다.')


{'question': '개통된 년도는 언제인가?',
 'contenxt': '한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다.',
 'answer': '1917년'}

In [35]:
contxt = '파이썬은 1991년 네덜란드계 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로, 플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically typed) 대화형 언어이다. 파이썬이라는 이름은 귀도가 좋아하는 코미디 〈Monty Python\'s Flying Circus〉에서 따온 것이다. '
inference_fn('파이썬을 발표한 사람은?', contxt)


{'question': '파이썬을 발표한 사람은?',
 'contenxt': "파이썬은 1991년 네덜란드계 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로, 플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically typed) 대화형 언어이다. 파이썬이라는 이름은 귀도가 좋아하는 코미디 〈Monty Python's Flying Circus〉에서 따온 것이다. ",
 'answer': '귀도 반 로섬'}